In [44]:
from transformers import pipeline, set_seed
generator = pipeline('text-generation', model='bigscience/bloomz-560m')
set_seed(42)

ERROR! Session/line number was not unique in database. History logging moved to new session 1883


ValueError: num_return_sequences has to be 1, but is 5 when doing greedy search.

In [149]:
import time
def bloomz(prompt,
          max_length = 32,
          top_k = 0,
          num_beams = 0,
          no_repeat_ngram_size = 2,
          top_p = 0.9,
          temperature=0.4,
          greedy_decoding = False,
          return_full_text = False):
    
    top_k = None if top_k == 0 else top_k
    do_sample = False if num_beams > 0 else not greedy_decoding
    num_beams = None if (greedy_decoding or num_beams == 0) else num_beams
    no_repeat_ngram_size = None if num_beams is None else no_repeat_ngram_size
    top_p = None if num_beams else top_p
    early_stopping = None if num_beams is None else num_beams > 0
    try:
        s = time.time()
        response = generator(
            prompt, 
            temperature=temperature, 
            max_new_tokens=max_length, 
            top_k = top_k,
            num_beams = num_beams,
            no_repeat_ngram_size = no_repeat_ngram_size,
            top_p = top_p,
            return_full_text = return_full_text
        )
        #print(response)
        proc_time = time.time()-s
        print(f"Processing time was {proc_time} seconds")
    except:
        print("Thinking...")
        return bloomz(prompt,
          max_length = max_length,
          top_k = top_k,
          num_beams = num_beams,
          no_repeat_ngram_size = no_repeat_ngram_size,
          top_p = top_p,
          temperature = temperature,
          greedy_decoding = greedy_decoding,
          return_full_text = return_full_text)
    return response

In [92]:
few_shot_prompt_v1 = lambda q, a, shots: f"""Clasificar incoherencia en respuestas a preguntas abiertas de alumnos de cuarto grado en matematicas. 
Se indicara con <E Incoherente /E> si la respuesta entre <R /R> es incoherente a la pregunta entre <P /P>.
Se indicara con <E Coherente /E> si la respuesta entre <R /R> es coherente a la pregunta entre <P /P>.
<P {shots[0][0]} /P>
<R {shots[0][1]} /R>
<E {shots[0][2]} /E>

<P {q} /P>. 
<R {a} /R>.
<E """

shots_v1 = [
    [
    "Agustín desea guardar unos dados de 1 cm cúbico en una caja que mide 5 cm de largo, 2 cm de ancho y 2 cm de alto. Su hermano dice que caben en la caja 15 dados.¿Es correcto lo que señala el hermano de Agustín? Explica con tus palabras.",
    "esta mal  por caben 20 dados", 
    "Coherente"
    ],
]
q, a = shots_v3[0][0], shots_v1[0][1]
few_shot_prompt_v1(q, a, shots_v1)

'Clasificar incoherencia en respuestas a preguntas abiertas de alumnos de cuarto grado en matematicas. \nSe indicara con <E Incoherente /E> si la respuesta entre <R /R> es incoherente a la pregunta entre <P /P>.\nSe indicara con <E Coherente /E> si la respuesta entre <R /R> es coherente a la pregunta entre <P /P>.\n<P Agustín desea guardar unos dados de 1 cm cúbico en una caja que mide 5 cm de largo, 2 cm de ancho y 2 cm de alto. Su hermano dice que caben en la caja 15 dados.¿Es correcto lo que señala el hermano de Agustín? Explica con tus palabras. /P>\n<R esta mal  por caben 20 dados /R>\n<E Coherente /E>\n\n<P Agustín desea guardar unos dados de 1 cm cúbico en una caja que mide 5 cm de largo, 2 cm de ancho y 2 cm de alto. Su hermano dice que caben en la caja 15 dados.¿Es correcto lo que señala el hermano de Agustín? Explica con tus palabras. /P>. \n<R esta mal  por caben 20 dados /R>.\n<E '

In [93]:
few_shot_prompt_v1(q, "2", shots_v3)

'Clasificar incoherencia en respuestas a preguntas abiertas de alumnos de cuarto grado en matematicas. \nSe indicara con <E Incoherente /E> si la respuesta entre <R /R> es incoherente a la pregunta entre <P /P>.\nSe indicara con <E Coherente /E> si la respuesta entre <R /R> es coherente a la pregunta entre <P /P>.\n<P Agustín desea guardar unos dados de 1 cm cúbico en una caja que mide 5 cm de largo, 2 cm de ancho y 2 cm de alto. Su hermano dice que caben en la caja 15 dados.¿Es correcto lo que señala el hermano de Agustín? Explica con tus palabras. /P>\n<R esta mal  por caben 20 dados /R>\n<E Coherente /E>\n\n<P Agustín desea guardar unos dados de 1 cm cúbico en una caja que mide 5 cm de largo, 2 cm de ancho y 2 cm de alto. Su hermano dice que caben en la caja 15 dados.¿Es correcto lo que señala el hermano de Agustín? Explica con tus palabras. /P>. \n<R 2 /R>.\n<E '

In [94]:
bloomz(few_shot_prompt_v1(q, "caca", shots_v3), max_length=8, temperature=0.4)

Processing time was 3.841254234313965 seconds


[{'generated_text': ' incoherente /E>'}]

In [16]:
import pandas as pd

In [17]:
test = pd.read_excel("data/test_task_C1.xlsx")

In [95]:
%%time
responses = []
for i, ix in enumerate(test.index[len(responses):]):
    q, a = test.loc[ix]["Q A".split()]
    o = bloomz(few_shot_prompt_v1(str(q).replace("\n", ""), str(a).replace("\n", ""), shots_v3), max_length=8, temperature=0.4)[0]['generated_text']
    responses.append([ix, o])
    print(100*(i+1)/test.shape[0], ix, o)

Processing time was 2.949749708175659 seconds
0.14771048744460857 0  incoherente
Processing time was 3.3622851371765137 seconds
0.29542097488921715 1  Coherente /E>
Processing time was 3.3031044006347656 seconds
0.4431314623338257 2  incoherente /E>
Processing time was 3.5706448554992676 seconds
0.5908419497784343 3  incoherente /E>
Processing time was 3.7046761512756348 seconds
0.7385524372230429 4  incoherente /E>
Processing time was 3.3610286712646484 seconds
0.8862629246676514 5  incoherente /E>
Processing time was 3.6631314754486084 seconds
1.03397341211226 6  incoherente /E>
Processing time was 3.8072617053985596 seconds
1.1816838995568686 7  Coherente /E>
Processing time was 3.8128409385681152 seconds
1.329394387001477 8  Coherente /E>
Processing time was 3.8493165969848633 seconds
1.4771048744460857 9  Coherente /E>
Processing time was 3.6588032245635986 seconds
1.6248153618906942 10  incoherente /E>
Processing time was 3.92549204826355 seconds
1.7725258493353029 11  incoherent

Processing time was 3.9069955348968506 seconds
14.771048744460856 99  Coherente /E>
Processing time was 4.052310228347778 seconds
14.918759231905465 100  incoherente /E>
Processing time was 3.2441182136535645 seconds
15.066469719350074 101  incoherente /E>
Processing time was 3.3046629428863525 seconds
15.214180206794682 102  incoherente /E>
Processing time was 3.6489343643188477 seconds
15.36189069423929 103  incoherente /E>
Processing time was 3.646320104598999 seconds
15.5096011816839 104  incoherente /E>
Processing time was 4.025346279144287 seconds
15.657311669128507 105  incoherente /E>
Processing time was 3.6013083457946777 seconds
15.805022156573116 106  incoherente /E>
Processing time was 2.8672637939453125 seconds
15.952732644017726 107  incoherente
Processing time was 3.6473193168640137 seconds
16.100443131462335 108  Coherente /E>
Processing time was 3.8796634674072266 seconds
16.248153618906944 109  incoherente /E>
Processing time was 4.073384761810303 seconds
16.395864106

Processing time was 3.404892683029175 seconds
29.098966026587888 196  incoherente /E>
Processing time was 3.5814340114593506 seconds
29.246676514032497 197  incoherente /E>
Processing time was 3.6493046283721924 seconds
29.394387001477106 198  Coherente /E>
Processing time was 3.159022092819214 seconds
29.542097488921712 199  incoherente /E>
Processing time was 3.304513454437256 seconds
29.68980797636632 200  Coherente /E>
Processing time was 3.983715772628784 seconds
29.83751846381093 201  incoherente /E>
Processing time was 3.810150623321533 seconds
29.98522895125554 202  incoherente /E>
Processing time was 3.9044876098632812 seconds
30.13293943870015 203  Coherente /E>
Processing time was 3.3438937664031982 seconds
30.280649926144758 204  Coherente /E>
Processing time was 3.2763688564300537 seconds
30.428360413589363 205  Coherente /E>
Processing time was 3.2084481716156006 seconds
30.576070901033972 206  incoherente /E>
Processing time was 3.638259172439575 seconds
30.7237813884785

Processing time was 3.2395834922790527 seconds
43.574593796159526 294  incoherente /E>
Processing time was 3.260484218597412 seconds
43.72230428360414 295  incoherente /E>
Processing time was 3.558641195297241 seconds
43.870014771048744 296  incoherente /E>
Processing time was 3.4529941082000732 seconds
44.01772525849335 297  incoherente /E>
Processing time was 3.9896092414855957 seconds
44.16543574593796 298  Coherente /E>
Processing time was 3.4273409843444824 seconds
44.31314623338257 299  Coherente /E>
Processing time was 3.8894436359405518 seconds
44.46085672082718 300  incoherente /E>
Processing time was 3.1991183757781982 seconds
44.608567208271786 301  incoherente /E>
Processing time was 3.7850818634033203 seconds
44.7562776957164 302  incoherente /E>
Processing time was 3.6141672134399414 seconds
44.903988183161005 303  Coherente /E>
Processing time was 3.145766019821167 seconds
45.05169867060561 304  incoherente
Processing time was 4.0478105545043945 seconds
45.19940915805022

Processing time was 4.624677896499634 seconds
57.90251107828656 391  incoherente /E>
Processing time was 5.470758438110352 seconds
58.050221565731164 392  Coherente /E>
Processing time was 4.1687633991241455 seconds
58.197932053175776 393  Coherente /E>
Processing time was 4.343899250030518 seconds
58.34564254062038 394  Coherente /E>
Processing time was 3.66103458404541 seconds
58.493353028064995 395  incoherente /E>
Processing time was 3.6933181285858154 seconds
58.6410635155096 396  incoherente /E>
Processing time was 3.8530805110931396 seconds
58.78877400295421 397  incoherente /E>
Processing time was 3.9794840812683105 seconds
58.93648449039882 398  Coherente /E>
Processing time was 3.539276123046875 seconds
59.084194977843424 399  incoherente /E>
Processing time was 3.6561129093170166 seconds
59.23190546528804 400  Coherente /E>
Processing time was 4.712942361831665 seconds
59.37961595273264 401  incoherente /E>
Processing time was 4.0694074630737305 seconds
59.527326440177255 40

Processing time was 4.036054849624634 seconds
72.3781388478582 489  Coherente /E>
Processing time was 3.1302905082702637 seconds
72.52584933530281 490  Coherente /E>
Processing time was 3.6732890605926514 seconds
72.67355982274742 491  Coherente /E>
Processing time was 4.440117597579956 seconds
72.82127031019202 492  Coherente /E>
Processing time was 3.5448548793792725 seconds
72.96898079763663 493  incoherente /E>
Processing time was 4.0871336460113525 seconds
73.11669128508125 494  incoherente /E>
Processing time was 3.641857862472534 seconds
73.26440177252584 495  incoherente /E>
Processing time was 3.890190362930298 seconds
73.41211225997046 496  Coherente /E>
Processing time was 3.8595471382141113 seconds
73.55982274741507 497  Coherente /E>
Processing time was 3.8018336296081543 seconds
73.70753323485968 498  incoherente /E>
Processing time was 3.9882020950317383 seconds
73.85524372230428 499  Coherente /E>
Processing time was 3.9501466751098633 seconds
74.00295420974889 500  Coh

Processing time was 4.138262033462524 seconds
86.85376661742984 587  Coherente /E>
Processing time was 4.000153303146362 seconds
87.00147710487444 588  incoherente /E>
Processing time was 3.114093780517578 seconds
87.14918759231905 589  incoherente
Processing time was 3.486050605773926 seconds
87.29689807976366 590  incoherente /E>
Processing time was 3.829838514328003 seconds
87.44460856720828 591  Coherente /E>
Processing time was 3.4431004524230957 seconds
87.59231905465288 592  incoherente /E>
Processing time was 3.9306812286376953 seconds
87.74002954209749 593  incoherente /E>
Processing time was 3.4934895038604736 seconds
87.8877400295421 594  Coherente /E>
Processing time was 3.411494255065918 seconds
88.0354505169867 595  incoherente /E>
Processing time was 3.4222779273986816 seconds
88.18316100443131 596  Coherente /E>
Processing time was 2.92985463142395 seconds
88.33087149187593 597  incoherente
Processing time was 4.071518421173096 seconds
88.47858197932054 598  incoherente

In [96]:
test_v1 = test.copy()
test_v1["bloomz"] = [x[1] for x in responses]
test_v1["bloomz-label"] = [1 if "incoherente" in x[1].lower() else 0 for x in responses]
test_v1

,id,Q_id,Q,A,label,bloomz,bloomz-label
0,85815,4581,¿Qué aprendiste de la actividad del viernes?,ha ver las cajas porque las cajas chicas eran ...,0,incoherente,1
1,64812,3535,¿Cómo creen ustedes que se reproducen las célu...,El proceso de división de una célula en dos cé...,0,Coherente /E>,0
2,84148,4491,¿ Qué frase para mejorar le darías tú a un am...,debes practicar los ejerccios,0,incoherente /E>,1
3,88565,4734,¿Qué aprendiste en la actividad del día viernes?,Nose porque no fui. A donde fueron ustedes😛😛😛😛😜,1,incoherente /E>,1
4,51688,2944,Diego compró 2 sopaipillas a $120 cada una. Pa...,las sopaipillas,1,incoherente /E>,1
...,...,...,...,...,...,...,...
672,54685,3109,Lorena dice que al multiplicar 3x0 le da como...,estamal poreque es 0,0,incoherente /E>,1
673,87324,4665,Alicia comió 2/9 de un chocolate. Ella dice qu...,esta mal Alicia por que la cantidad es 7/8 de...,0,Coherente /E>,0
674,73798,3946,El cuarto básico visitará el zoológico. Si son...,nose\n,1,incoherente /E>,1
675,95242,5093,Durante la sesión de conecta ideas María Jesús...,"es correcto, porque yo lo \nmultiplique por 4...",0,No,0


In [97]:
test_v1.to_excel("bloomz_label_v1.xlsx")

In [98]:
from sklearn.metrics import classification_report

In [99]:
print(classification_report(test_v1["label"], test_v1["bloomz-label"], digits=4))

              precision    recall  f1-score   support

           0     0.8586    0.4824    0.6178       541
           1     0.2493    0.6838    0.3654       136

    accuracy                         0.5229       677
   macro avg     0.5539    0.5831    0.4916       677
weighted avg     0.7362    0.5229    0.5671       677



In [100]:
train = pd.read_excel("data/train_task_C1.xlsx")

In [104]:
q, a, label = train.sample(1).iloc[0]["Q A label".split()]
o = bloomz(few_shot_prompt_v1(str(q).replace("\n", ""), str(a).replace("\n", ""), shots_v3), max_length=8, temperature=0.4)[0]['generated_text']
bloomz_label = 1 if "incoherente" in o.lower() else 0
print(q, a, bloomz_label, label)

Processing time was 3.2194786071777344 seconds
María y su esposo ayer cocinaron una tortilla, la dividieron en 6 partes iguales. María comió 2/6 y su esposo 3/6. ¿Qué fracción de la tortilla quedo?  quedo un pedaso de tortilla
y la ultima quedo a y la tortilla 1 0


In [106]:
few_shot_prompt_v2 = lambda q, a, shots: f"""Clasificar incoherencia en respuestas a preguntas abiertas de alumnos de cuarto grado en matematicas. 
Se indicara con <E Incoherente /E> si la respuesta entre <R /R> es incoherente a la pregunta entre <P /P>.
Se indicara con <E Coherente /E> si la respuesta entre <R /R> es coherente a la pregunta entre <P /P>.
<P {shots[0][0]} /P>
<R {shots[0][1]} /R>
<E {shots[0][2]} /E>

<P {shots[1][0]} /P>
<R {shots[1][1]} /R>
<E {shots[1][2]} /E>

<P {q} /P>. 
<R {a} /R>.
<E """

shots_v2 = [
    [
    "Agustín desea guardar unos dados de 1 cm cúbico en una caja que mide 5 cm de largo, 2 cm de ancho y 2 cm de alto. Su hermano dice que caben en la caja 15 dados.¿Es correcto lo que señala el hermano de Agustín? Explica con tus palabras.",
    "esta mal  por caben 20 dados", 
    "Coherente"
    ],
    [
    "María y su esposo ayer cocinaron una tortilla, la dividieron en 6 partes iguales. María comió 2/6 y su esposo 3/6. ¿Qué fracción de la tortilla quedo?",
    "quedo un pedaso de tortilla y la ultima quedo a y la tortilla",
    "Coherente"
    ]
]
q, a = shots_v2[0][0], shots_v2[0][1]
few_shot_prompt_v2(q, a, shots_v2)

'Clasificar incoherencia en respuestas a preguntas abiertas de alumnos de cuarto grado en matematicas. \nSe indicara con <E Incoherente /E> si la respuesta entre <R /R> es incoherente a la pregunta entre <P /P>.\nSe indicara con <E Coherente /E> si la respuesta entre <R /R> es coherente a la pregunta entre <P /P>.\n<P Agustín desea guardar unos dados de 1 cm cúbico en una caja que mide 5 cm de largo, 2 cm de ancho y 2 cm de alto. Su hermano dice que caben en la caja 15 dados.¿Es correcto lo que señala el hermano de Agustín? Explica con tus palabras. /P>\n<R esta mal  por caben 20 dados /R>\n<E Coherente /E>\n\n<P María y su esposo ayer cocinaron una tortilla, la dividieron en 6 partes iguales. María comió 2/6 y su esposo 3/6. ¿Qué fracción de la tortilla quedo? /P>\n<R quedo un pedaso de tortilla y la ultima quedo a y la tortilla /R>\n<E Coherente /E>\n\n<P Agustín desea guardar unos dados de 1 cm cúbico en una caja que mide 5 cm de largo, 2 cm de ancho y 2 cm de alto. Su hermano dice 

In [107]:
bloomz(few_shot_prompt_v2(q, "caca", shots_v2), max_length=8, temperature=0.4)

Processing time was 5.118091821670532 seconds


[{'generated_text': ' incoherente /E>'}]

In [108]:
%%time
responses = []
for i, ix in enumerate(test.index[len(responses):]):
    q, a = test.loc[ix]["Q A".split()]
    o = bloomz(few_shot_prompt_v2(str(q).replace("\n", ""), str(a).replace("\n", ""), shots_v2), max_length=8, temperature=0.4)[0]['generated_text']
    responses.append([ix, o])
    print(100*(i+1)/test.shape[0], ix, o)

Processing time was 4.9089202880859375 seconds
0.14771048744460857 0  Coherente /E>
Processing time was 4.601450443267822 seconds
0.29542097488921715 1  Coherente /E>
Processing time was 4.699859619140625 seconds
0.4431314623338257 2  Coherente /E>
Processing time was 4.729254245758057 seconds
0.5908419497784343 3  incoherente /E>
Processing time was 4.978672742843628 seconds
0.7385524372230429 4  Coherente /E>
Processing time was 4.439018726348877 seconds
0.8862629246676514 5  incoherente /E>
Processing time was 5.113434314727783 seconds
1.03397341211226 6  Coherente /E>
Processing time was 5.251093626022339 seconds
1.1816838995568686 7  Coherente /E>
Processing time was 5.108225107192993 seconds
1.329394387001477 8  Coherente /E>
Processing time was 5.169937372207642 seconds
1.4771048744460857 9  Coherente /E>
Processing time was 4.67028021812439 seconds
1.6248153618906942 10  incoherente /E>
Processing time was 5.159468173980713 seconds
1.7725258493353029 11  Coherente /E>
Processin

Processing time was 5.591104984283447 seconds
14.918759231905465 100  Coherente /E>
Processing time was 4.952245712280273 seconds
15.066469719350074 101  incoherente /E>
Processing time was 5.015502452850342 seconds
15.214180206794682 102  Coherente /E>
Processing time was 5.22049617767334 seconds
15.36189069423929 103  incoherente /E>
Processing time was 5.146515130996704 seconds
15.5096011816839 104  Coherente /E>
Processing time was 4.986016273498535 seconds
15.657311669128507 105  Coherente /E>
Processing time was 3.8948159217834473 seconds
15.805022156573116 106  incoherente
Processing time was 4.753687143325806 seconds
15.952732644017726 107  incoherente /E>
Processing time was 5.001827239990234 seconds
16.100443131462335 108  Coherente /E>
Processing time was 5.233484745025635 seconds
16.248153618906944 109  Coherente /E>
Processing time was 5.007982015609741 seconds
16.39586410635155 110  Coherente /E>
Processing time was 3.984527349472046 seconds
16.54357459379616 111  incoher

Processing time was 5.073003530502319 seconds
29.394387001477106 198  Coherente /E>
Processing time was 4.7184624671936035 seconds
29.542097488921712 199  Coherente /E>
Processing time was 4.740493059158325 seconds
29.68980797636632 200  Coherente /E>
Processing time was 5.01844334602356 seconds
29.83751846381093 201  Coherente /E>
Processing time was 5.35545539855957 seconds
29.98522895125554 202  Coherente /E>
Processing time was 5.167161703109741 seconds
30.13293943870015 203  Coherente /E>
Processing time was 4.719228267669678 seconds
30.280649926144758 204  Coherente /E>
Processing time was 4.6220996379852295 seconds
30.428360413589363 205  Coherente /E>
Processing time was 4.338106155395508 seconds
30.576070901033972 206  incoherente /E>
Processing time was 4.637979745864868 seconds
30.72378138847858 207  incoherente /E>
Processing time was 4.548677206039429 seconds
30.87149187592319 208  Coherente /E>
Processing time was 4.636427879333496 seconds
31.0192023633678 209  incoherent

Processing time was 5.026991128921509 seconds
43.870014771048744 296  incoherente /E>
Processing time was 4.414291143417358 seconds
44.01772525849335 297  incoherente /E>
Processing time was 5.484450340270996 seconds
44.16543574593796 298  Coherente /E>
Processing time was 4.876113414764404 seconds
44.31314623338257 299  Coherente /E>
Processing time was 5.2120726108551025 seconds
44.46085672082718 300  Coherente /E>
Processing time was 4.734834909439087 seconds
44.608567208271786 301  incoherente /E>
Processing time was 5.329931974411011 seconds
44.7562776957164 302  Coherente /E>
Processing time was 5.044113636016846 seconds
44.903988183161005 303  Coherente /E>
Processing time was 4.180781364440918 seconds
45.05169867060561 304  incoherente
Processing time was 5.473516941070557 seconds
45.19940915805022 305  Coherente /E>
Processing time was 5.414667129516602 seconds
45.34711964549483 306  Coherente /E>
Processing time was 5.511775970458984 seconds
45.49483013293944 307  Coherente /

Processing time was 5.809373378753662 seconds
58.34564254062038 394  Coherente /E>
Processing time was 5.06489896774292 seconds
58.493353028064995 395  Coherente /E>
Processing time was 4.125205039978027 seconds
58.6410635155096 396  incoherente
Processing time was 4.608139753341675 seconds
58.78877400295421 397  incoherente /E>
Processing time was 5.057228326797485 seconds
58.93648449039882 398  Coherente /E>
Processing time was 4.715746164321899 seconds
59.084194977843424 399  incoherente /E>
Processing time was 5.181687355041504 seconds
59.23190546528804 400  Coherente /E>
Processing time was 5.438091039657593 seconds
59.37961595273264 401  Coherente /E>
Processing time was 4.56164026260376 seconds
59.527326440177255 402  Coherente /E>
Processing time was 4.728941440582275 seconds
59.67503692762186 403  Coherente /E>
Processing time was 5.098125457763672 seconds
59.822747415066466 404  Coherente /E>
Processing time was 4.752026557922363 seconds
59.97045790251108 405  Coherente /E>
P

Processing time was 4.285138845443726 seconds
72.96898079763663 493  incoherente /E>
Processing time was 5.003622055053711 seconds
73.11669128508125 494  Coherente /E>
Processing time was 4.822125434875488 seconds
73.26440177252584 495  Coherente /E>
Processing time was 5.113850831985474 seconds
73.41211225997046 496  Coherente /E>
Processing time was 4.990108013153076 seconds
73.55982274741507 497  Coherente /E>
Processing time was 4.870744943618774 seconds
73.70753323485968 498  Coherente /E>
Processing time was 4.96371865272522 seconds
73.85524372230428 499  Coherente /E>
Processing time was 5.038317918777466 seconds
74.00295420974889 500  Coherente /E>
Processing time was 4.369309663772583 seconds
74.1506646971935 501  Coherente /E>
Processing time was 4.313428640365601 seconds
74.2983751846381 502  Coherente /E>
Processing time was 4.849783420562744 seconds
74.44608567208272 503  Coherente /E>
Processing time was 5.010020732879639 seconds
74.59379615952733 504  Coherente /E>
Proce

Processing time was 4.496289968490601 seconds
87.59231905465288 592  Coherente /E>
Processing time was 5.0105438232421875 seconds
87.74002954209749 593  Coherente /E>
Processing time was 4.4872846603393555 seconds
87.8877400295421 594  Coherente /E>
Processing time was 4.557956695556641 seconds
88.0354505169867 595  incoherente /E>
Processing time was 4.252261400222778 seconds
88.18316100443131 596  Coherente /E>
Processing time was 4.40219783782959 seconds
88.33087149187593 597  Coherente /E>
Processing time was 5.107792139053345 seconds
88.47858197932054 598  incoherente /E>
Processing time was 4.424741268157959 seconds
88.62629246676514 599  Coherente /E>
Processing time was 5.004302740097046 seconds
88.77400295420975 600  Coherente /E>
Processing time was 4.470315456390381 seconds
88.92171344165436 601  Coherente /E>
Processing time was 4.392818450927734 seconds
89.06942392909896 602  Coherente /E>
Processing time was 4.597944021224976 seconds
89.21713441654357 603  Coherente /E>
P

In [109]:
test_v2 = test.copy()
test_v2["bloomz"] = [x[1] for x in responses]
test_v2["bloomz-label"] = [1 if "incoherente" in x[1].lower() else 0 for x in responses]
test_v2

,id,Q_id,Q,A,label,bloomz,bloomz-label
0,85815,4581,¿Qué aprendiste de la actividad del viernes?,ha ver las cajas porque las cajas chicas eran ...,0,Coherente /E>,0
1,64812,3535,¿Cómo creen ustedes que se reproducen las célu...,El proceso de división de una célula en dos cé...,0,Coherente /E>,0
2,84148,4491,¿ Qué frase para mejorar le darías tú a un am...,debes practicar los ejerccios,0,Coherente /E>,0
3,88565,4734,¿Qué aprendiste en la actividad del día viernes?,Nose porque no fui. A donde fueron ustedes😛😛😛😛😜,1,incoherente /E>,1
4,51688,2944,Diego compró 2 sopaipillas a $120 cada una. Pa...,las sopaipillas,1,Coherente /E>,0
...,...,...,...,...,...,...,...
672,54685,3109,Lorena dice que al multiplicar 3x0 le da como...,estamal poreque es 0,0,incoherente /E>,1
673,87324,4665,Alicia comió 2/9 de un chocolate. Ella dice qu...,esta mal Alicia por que la cantidad es 7/8 de...,0,Coherente /E>,0
674,73798,3946,El cuarto básico visitará el zoológico. Si son...,nose\n,1,incoherente /E>,1
675,95242,5093,Durante la sesión de conecta ideas María Jesús...,"es correcto, porque yo lo \nmultiplique por 4...",0,Coherente /E>,0


In [110]:
test_v2.to_excel("bloomz_label_v2.xlsx")

In [111]:
print(classification_report(test_v2["label"], test_v2["bloomz-label"], digits=4))

              precision    recall  f1-score   support

           0     0.8785    0.7616    0.8158       541
           1     0.3798    0.5809    0.4593       136

    accuracy                         0.7253       677
   macro avg     0.6291    0.6712    0.6376       677
weighted avg     0.7783    0.7253    0.7442       677



In [138]:
q, a, label = train.sample(1).iloc[0]["Q A label".split()]
o = bloomz(few_shot_prompt_v2(str(q).replace("\n", ""), str(a).replace("\n", ""), shots_v2), max_length=8, temperature=0.4)[0]['generated_text']
bloomz_label = 1 if "incoherente" in o.lower() else 0
print(q, a, bloomz_label, label)

Processing time was 5.035702466964722 seconds
Claudio tiene 12 lápices para repartir en cantidades iguales en 3 estuches, para hacerlo, su amiga Paula le dice que deberá poner 6 lápices en cada estuche y así no le sobrará ninguno. ¿Está en lo correcto la amiga de Claudio? (Explica con tus palabras la respuesta) paula no esta en lo correcto porque me pide en mas estuches 1 0


In [139]:
few_shot_prompt_v3 = lambda q, a, shots: f"""Clasificar incoherencia en respuestas a preguntas abiertas de alumnos de cuarto grado en matematicas. 
Se indicara con <E Incoherente /E> si la respuesta entre <R /R> es incoherente a la pregunta entre <P /P>.
Se indicara con <E Coherente /E> si la respuesta entre <R /R> es coherente a la pregunta entre <P /P>.
<P {shots[0][0]} /P>
<R {shots[0][1]} /R>
<E {shots[0][2]} /E>

<P {shots[1][0]} /P>
<R {shots[1][1]} /R>
<E {shots[1][2]} /E>

<P {shots[2][0]} /P>
<R {shots[2][1]} /R>
<E {shots[2][2]} /E>

<P {q} /P>. 
<R {a} /R>.
<E """

shots_v3 = [
    [
    "Agustín desea guardar unos dados de 1 cm cúbico en una caja que mide 5 cm de largo, 2 cm de ancho y 2 cm de alto. Su hermano dice que caben en la caja 15 dados.¿Es correcto lo que señala el hermano de Agustín? Explica con tus palabras.",
    "esta mal  por caben 20 dados", 
    "Coherente"
    ],
    [
    "María y su esposo ayer cocinaron una tortilla, la dividieron en 6 partes iguales. María comió 2/6 y su esposo 3/6. ¿Qué fracción de la tortilla quedo?",
    "quedo un pedaso de tortilla y la ultima quedo a y la tortilla",
    "Coherente"
    ],
    [
    "Claudio tiene 12 lápices para repartir en cantidades iguales en 3 estuches, para hacerlo, su amiga Paula le dice que deberá poner 6 lápices en cada estuche y así no le sobrará ninguno. ¿Está en lo correcto la amiga de Claudio? (Explica con tus palabras la respuesta)",
    "paula no esta en lo correcto porque me pide en mas estuches",
    "Coherente"
    ]
]
q, a = shots_v3[0][0], shots_v3[0][1]
few_shot_prompt_v3(q, a, shots_v3)

'Clasificar incoherencia en respuestas a preguntas abiertas de alumnos de cuarto grado en matematicas. \nSe indicara con <E Incoherente /E> si la respuesta entre <R /R> es incoherente a la pregunta entre <P /P>.\nSe indicara con <E Coherente /E> si la respuesta entre <R /R> es coherente a la pregunta entre <P /P>.\n<P Agustín desea guardar unos dados de 1 cm cúbico en una caja que mide 5 cm de largo, 2 cm de ancho y 2 cm de alto. Su hermano dice que caben en la caja 15 dados.¿Es correcto lo que señala el hermano de Agustín? Explica con tus palabras. /P>\n<R esta mal  por caben 20 dados /R>\n<E Coherente /E>\n\n<P María y su esposo ayer cocinaron una tortilla, la dividieron en 6 partes iguales. María comió 2/6 y su esposo 3/6. ¿Qué fracción de la tortilla quedo? /P>\n<R quedo un pedaso de tortilla y la ultima quedo a y la tortilla /R>\n<E Coherente /E>\n\n<P Claudio tiene 12 lápices para repartir en cantidades iguales en 3 estuches, para hacerlo, su amiga Paula le dice que deberá poner 

In [140]:
bloomz(few_shot_prompt_v3(q, "caca", shots_v3), max_length=8, temperature=0.4)

Processing time was 6.502973556518555 seconds


[{'generated_text': ' incoherente /E>'}]

In [141]:
%%time
responses = []
for i, ix in enumerate(test.index[len(responses):]):
    q, a = test.loc[ix]["Q A".split()]
    o = bloomz(few_shot_prompt_v3(str(q).replace("\n", ""), str(a).replace("\n", ""), shots_v3), max_length=8, temperature=0.4)[0]['generated_text']
    responses.append([ix, o])
    print(100*(i+1)/test.shape[0], ix, o)

Processing time was 6.176124572753906 seconds
0.14771048744460857 0  Coherente /E>
Processing time was 6.0213377475738525 seconds
0.29542097488921715 1  Coherente /E>
Processing time was 5.533263206481934 seconds
0.4431314623338257 2  incoherente
Processing time was 5.6430463790893555 seconds
0.5908419497784343 3  incoherente /E>
Processing time was 5.476691484451294 seconds
0.7385524372230429 4  incoherente /E
Processing time was 4.744377136230469 seconds
0.8862629246676514 5  incoherente /E
Processing time was 5.797467470169067 seconds
1.03397341211226 6  Coherente /E>
Processing time was 5.775346517562866 seconds
1.1816838995568686 7  Coherente /E>
Processing time was 5.926839590072632 seconds
1.329394387001477 8  incoherente /E>
Processing time was 5.666655540466309 seconds
1.4771048744460857 9  incoherente /E>
Processing time was 4.794193983078003 seconds
1.6248153618906942 10  incoherente
Processing time was 5.94219708442688 seconds
1.7725258493353029 11  Coherente /E>
Processing

Processing time was 6.825816869735718 seconds
14.918759231905465 100  Coherente /E>
Processing time was 5.700742959976196 seconds
15.066469719350074 101  incoherente /E>
Processing time was 5.602271556854248 seconds
15.214180206794682 102  incoherente /E
Processing time was 6.025132656097412 seconds
15.36189069423929 103  incoherente /E
Processing time was 6.092030048370361 seconds
15.5096011816839 104  Coherente /E>
Processing time was 6.504186391830444 seconds
15.657311669128507 105  Coherente /E>
Processing time was 6.124917507171631 seconds
15.805022156573116 106  Coherente /E>
Processing time was 6.050392150878906 seconds
15.952732644017726 107  incoherente /E>
Processing time was 6.632091999053955 seconds
16.100443131462335 108  Coherente /E>
Processing time was 6.9501965045928955 seconds
16.248153618906944 109  Coherente /E>
Processing time was 7.166489601135254 seconds
16.39586410635155 110  Coherente /E>
Processing time was 6.028271198272705 seconds
16.54357459379616 111  inco

Processing time was 7.039963006973267 seconds
29.394387001477106 198  Coherente /E>
Processing time was 5.835508108139038 seconds
29.542097488921712 199  Coherente /E>
Processing time was 6.166388988494873 seconds
29.68980797636632 200  Coherente /E>
Processing time was 7.054606199264526 seconds
29.83751846381093 201  Coherente /E>
Processing time was 7.467576742172241 seconds
29.98522895125554 202  Coherente /E>
Processing time was 6.517156362533569 seconds
30.13293943870015 203  Coherente /E>
Processing time was 6.153445720672607 seconds
30.280649926144758 204  Coherente /E>
Processing time was 5.825024843215942 seconds
30.428360413589363 205  Coherente /E>
Processing time was 5.255963087081909 seconds
30.576070901033972 206  incoherente /E
Processing time was 5.986511707305908 seconds
30.72378138847858 207  incoherente /E>
Processing time was 6.597589492797852 seconds
30.87149187592319 208  Coherente /E>
Processing time was 5.827843904495239 seconds
31.0192023633678 209  incoherente

Processing time was 6.154083013534546 seconds
44.01772525849335 297  incoherente /E>
Processing time was 6.936259746551514 seconds
44.16543574593796 298  Coherente /E>
Processing time was 6.688247203826904 seconds
44.31314623338257 299  Coherente /E>
Processing time was 6.908854961395264 seconds
44.46085672082718 300  Coherente /E>
Processing time was 5.610117197036743 seconds
44.608567208271786 301  incoherente /E
Processing time was 6.610690593719482 seconds
44.7562776957164 302  Coherente /E>
Processing time was 6.571115970611572 seconds
44.903988183161005 303  Coherente /E>
Processing time was 6.1877264976501465 seconds
45.05169867060561 304  incoherente
Processing time was 6.826076984405518 seconds
45.19940915805022 305  Coherente /E>
Processing time was 7.281431674957275 seconds
45.34711964549483 306  Coherente /E>
Processing time was 6.841788291931152 seconds
45.49483013293944 307  Coherente /E>
Processing time was 5.753158807754517 seconds
45.64254062038405 308  incoherente /E


Processing time was 6.139249086380005 seconds
58.493353028064995 395  incoherente /E
Processing time was 5.590194225311279 seconds
58.6410635155096 396  incoherente
Processing time was 5.388422250747681 seconds
58.78877400295421 397  incoherente
Processing time was 6.891024112701416 seconds
58.93648449039882 398  Coherente /E>
Processing time was 5.952491998672485 seconds
59.084194977843424 399  incoherente /E
Processing time was 6.639209270477295 seconds
59.23190546528804 400  Coherente /E>
Processing time was 7.006863117218018 seconds
59.37961595273264 401  Coherente /E>
Processing time was 6.299293041229248 seconds
59.527326440177255 402  Coherente /E>
Processing time was 6.510610818862915 seconds
59.67503692762186 403  incoherente /E>
Processing time was 7.016833543777466 seconds
59.822747415066466 404  Coherente /E>
Processing time was 6.522528409957886 seconds
59.97045790251108 405  Coherente /E>
Processing time was 6.5648581981658936 seconds
60.118168389955684 406  Coherente /E>

Processing time was 6.8790788650512695 seconds
73.11669128508125 494  Coherente /E>
Processing time was 6.774738788604736 seconds
73.26440177252584 495  Coherente /E>
Processing time was 6.627071142196655 seconds
73.41211225997046 496  Coherente /E>
Processing time was 6.547085762023926 seconds
73.55982274741507 497  Coherente /E>
Processing time was 6.267130136489868 seconds
73.70753323485968 498  Coherente /E>
Processing time was 6.450204610824585 seconds
73.85524372230428 499  Coherente /E>
Processing time was 6.50995659828186 seconds
74.00295420974889 500  Coherente /E>
Processing time was 5.752484083175659 seconds
74.1506646971935 501  Coherente /E>
Processing time was 5.730127334594727 seconds
74.2983751846381 502  Coherente /E>
Processing time was 7.80210280418396 seconds
74.44608567208272 503  Coherente /E>
Processing time was 6.831444978713989 seconds
74.59379615952733 504  Coherente /E>
Processing time was 6.6480772495269775 seconds
74.74150664697194 505  Coherente /E>
Proces

Processing time was 6.829041957855225 seconds
87.74002954209749 593  Coherente /E>
Processing time was 6.022610187530518 seconds
87.8877400295421 594  Coherente /E>
Processing time was 5.589781761169434 seconds
88.0354505169867 595  incoherente /E>
Processing time was 5.236424922943115 seconds
88.18316100443131 596  incoherente /E>
Processing time was 5.303483724594116 seconds
88.33087149187593 597  Coherente /E>
Processing time was 6.124940395355225 seconds
88.47858197932054 598  incoherente /E>
Processing time was 5.571983337402344 seconds
88.62629246676514 599  incoherente /E>
Processing time was 6.179336309432983 seconds
88.77400295420975 600  Coherente /E>
Processing time was 5.803346157073975 seconds
88.92171344165436 601  Coherente /E>
Processing time was 5.737962245941162 seconds
89.06942392909896 602  Coherente /E>
Processing time was 5.652737855911255 seconds
89.21713441654357 603  incoherente
Processing time was 6.427066326141357 seconds
89.36484490398819 604  Coherente /E>


In [142]:
test_v3 = test.copy()
test_v3["bloomz"] = [x[1] for x in responses]
test_v3["bloomz-label"] = [1 if "incoherente" in x[1].lower() else 0 for x in responses]
test_v3

,id,Q_id,Q,A,label,bloomz,bloomz-label
0,85815,4581,¿Qué aprendiste de la actividad del viernes?,ha ver las cajas porque las cajas chicas eran ...,0,Coherente /E>,0
1,64812,3535,¿Cómo creen ustedes que se reproducen las célu...,El proceso de división de una célula en dos cé...,0,Coherente /E>,0
2,84148,4491,¿ Qué frase para mejorar le darías tú a un am...,debes practicar los ejerccios,0,incoherente,1
3,88565,4734,¿Qué aprendiste en la actividad del día viernes?,Nose porque no fui. A donde fueron ustedes😛😛😛😛😜,1,incoherente /E>,1
4,51688,2944,Diego compró 2 sopaipillas a $120 cada una. Pa...,las sopaipillas,1,incoherente /E,1
...,...,...,...,...,...,...,...
672,54685,3109,Lorena dice que al multiplicar 3x0 le da como...,estamal poreque es 0,0,incoherente /E,1
673,87324,4665,Alicia comió 2/9 de un chocolate. Ella dice qu...,esta mal Alicia por que la cantidad es 7/8 de...,0,Coherente /E>,0
674,73798,3946,El cuarto básico visitará el zoológico. Si son...,nose\n,1,incoherente /E,1
675,95242,5093,Durante la sesión de conecta ideas María Jesús...,"es correcto, porque yo lo \nmultiplique por 4...",0,No,0


In [143]:
test_v3.to_excel("bloomz_label_v3.xlsx")

In [144]:
print(classification_report(test_v3["label"], test_v3["bloomz-label"], digits=4))

              precision    recall  f1-score   support

           0     0.8998    0.7301    0.8061       541
           1     0.3866    0.6765    0.4920       136

    accuracy                         0.7194       677
   macro avg     0.6432    0.7033    0.6491       677
weighted avg     0.7967    0.7194    0.7430       677



In [145]:
q, a, label = train.sample(1).iloc[0]["Q A label".split()]
o = bloomz(few_shot_prompt_v3(str(q).replace("\n", ""), str(a).replace("\n", ""), shots_v3), max_length=8, temperature=0.4)[0]['generated_text']
bloomz_label = 1 if "incoherente" in o.lower() else 0
print(q, a, bloomz_label, label)

Processing time was 8.4818274974823 seconds
Francisco y Camila tienen una colección de figuras de acción de su serie favorita, Francisco tiene 4/8 de la colección y Camila tiene 2/8, si del total de figuras que Francisco y Camila tienen pierden 5/8. ¿Cuántas figuras de la colección le quedan? (Redacta adecuadamente tu respuesta)  ESTA BUENA 0 1


In [146]:
few_shot_prompt_v4 = lambda q, a, shots: f"""Clasificar incoherencia en respuestas a preguntas abiertas de alumnos de cuarto grado en matematicas. 
Se indicara con <E Incoherente /E> si la respuesta entre <R /R> es incoherente a la pregunta entre <P /P>.
Se indicara con <E Coherente /E> si la respuesta entre <R /R> es coherente a la pregunta entre <P /P>.
<P {shots[0][0]} /P>
<R {shots[0][1]} /R>
<E {shots[0][2]} /E>

<P {shots[1][0]} /P>
<R {shots[1][1]} /R>
<E {shots[1][2]} /E>

<P {shots[2][0]} /P>
<R {shots[2][1]} /R>
<E {shots[2][2]} /E>

<P {shots[3][0]} /P>
<R {shots[3][1]} /R>
<E {shots[3][2]} /E>

<P {q} /P>. 
<R {a} /R>.
<E """

shots_v4 = [
    [
    "Agustín desea guardar unos dados de 1 cm cúbico en una caja que mide 5 cm de largo, 2 cm de ancho y 2 cm de alto. Su hermano dice que caben en la caja 15 dados.¿Es correcto lo que señala el hermano de Agustín? Explica con tus palabras.",
    "esta mal  por caben 20 dados", 
    "Coherente"
    ],
    [
    "María y su esposo ayer cocinaron una tortilla, la dividieron en 6 partes iguales. María comió 2/6 y su esposo 3/6. ¿Qué fracción de la tortilla quedo?",
    "quedo un pedaso de tortilla y la ultima quedo a y la tortilla",
    "Coherente"
    ],
    [
    "Claudio tiene 12 lápices para repartir en cantidades iguales en 3 estuches, para hacerlo, su amiga Paula le dice que deberá poner 6 lápices en cada estuche y así no le sobrará ninguno. ¿Está en lo correcto la amiga de Claudio? (Explica con tus palabras la respuesta)",
    "paula no esta en lo correcto porque me pide en mas estuches",
    "Coherente"
    ],
    [
    "Francisco y Camila tienen una colección de figuras de acción de su serie favorita, Francisco tiene 4/8 de la colección y Camila tiene 2/8, si del total de figuras que Francisco y Camila tienen pierden 5/8. ¿Cuántas figuras de la colección le quedan? (Redacta adecuadamente tu respuesta)",
    "ESTA BUENA",
    "Incoherente"
    ]
]
q, a = shots_v4[0][0], shots_v4[0][1]
few_shot_prompt_v4(q, a, shots_v4)

'Clasificar incoherencia en respuestas a preguntas abiertas de alumnos de cuarto grado en matematicas. \nSe indicara con <E Incoherente /E> si la respuesta entre <R /R> es incoherente a la pregunta entre <P /P>.\nSe indicara con <E Coherente /E> si la respuesta entre <R /R> es coherente a la pregunta entre <P /P>.\n<P Agustín desea guardar unos dados de 1 cm cúbico en una caja que mide 5 cm de largo, 2 cm de ancho y 2 cm de alto. Su hermano dice que caben en la caja 15 dados.¿Es correcto lo que señala el hermano de Agustín? Explica con tus palabras. /P>\n<R esta mal  por caben 20 dados /R>\n<E Coherente /E>\n\n<P María y su esposo ayer cocinaron una tortilla, la dividieron en 6 partes iguales. María comió 2/6 y su esposo 3/6. ¿Qué fracción de la tortilla quedo? /P>\n<R quedo un pedaso de tortilla y la ultima quedo a y la tortilla /R>\n<E Coherente /E>\n\n<P Claudio tiene 12 lápices para repartir en cantidades iguales en 3 estuches, para hacerlo, su amiga Paula le dice que deberá poner 

In [148]:
bloomz(few_shot_prompt_v4(q, "caca", shots_v4), max_length=8, temperature=0.4)

Processing time was 183.7700958251953 seconds


[{'generated_text': ' incoherente /E>'}]

In [151]:
%%time
responses = []
for i, ix in enumerate(test.index[len(responses):]):
    q, a = test.loc[ix]["Q A".split()]
    o = bloomz(few_shot_prompt_v4(str(q).replace("\n", ""), str(a).replace("\n", ""), shots_v4), max_length=8, temperature=0.4)[0]['generated_text']
    responses.append([ix, o])
    print(100*(i+1)/test.shape[0], ix, o)

Processing time was 8.39267349243164 seconds
0.14771048744460857 0  Coherente /E>
Processing time was 7.824557781219482 seconds
0.29542097488921715 1  Coherente /E>
Processing time was 8.177438735961914 seconds
0.4431314623338257 2  Coherente /E>
Processing time was 7.782313346862793 seconds
0.5908419497784343 3  Coherente /E>
Processing time was 8.097743034362793 seconds
0.7385524372230429 4  Coherente /E>
Processing time was 7.451395034790039 seconds
0.8862629246676514 5  Coherente /E>
Processing time was 7.481580495834351 seconds
1.03397341211226 6  Coherente /E>
Processing time was 7.901330947875977 seconds
1.1816838995568686 7  Coherente /E>
Processing time was 8.50911569595337 seconds
1.329394387001477 8  incoherente /E>
Processing time was 8.033668518066406 seconds
1.4771048744460857 9  Coherente /E>
Processing time was 7.7169530391693115 seconds
1.6248153618906942 10  Coherente /E>
Processing time was 8.38101077079773 seconds
1.7725258493353029 11  Coherente /E>
Processing time

Processing time was 9.490953207015991 seconds
14.918759231905465 100  Coherente /E>
Processing time was 7.434974670410156 seconds
15.066469719350074 101  incoherente /E>
Processing time was 7.778784990310669 seconds
15.214180206794682 102  Coherente /E>
Processing time was 8.345443964004517 seconds
15.36189069423929 103  Coherente /E>
Processing time was 8.573689222335815 seconds
15.5096011816839 104  Coherente /E>
Processing time was 9.67305326461792 seconds
15.657311669128507 105  Coherente /E>
Processing time was 7.9733498096466064 seconds
15.805022156573116 106  Coherente /E>
Processing time was 6.998811721801758 seconds
15.952732644017726 107  Coherente /E>
Processing time was 7.441205024719238 seconds
16.100443131462335 108  Coherente /E>
Processing time was 8.113237619400024 seconds
16.248153618906944 109  Incoherente /E>
Processing time was 8.39263129234314 seconds
16.39586410635155 110  Coherente /E>
Processing time was 7.1815290451049805 seconds
16.54357459379616 111  Coheren

Processing time was 7.489305734634399 seconds
29.394387001477106 198  Coherente /E>
Processing time was 7.268808603286743 seconds
29.542097488921712 199  Coherente /E>
Processing time was 7.1048743724823 seconds
29.68980797636632 200  Coherente /E>
Processing time was 7.940813302993774 seconds
29.83751846381093 201  Coherente /E>
Processing time was 7.73673152923584 seconds
29.98522895125554 202  Coherente /E>
Processing time was 7.681572914123535 seconds
30.13293943870015 203  Coherente /E>
Processing time was 6.960525274276733 seconds
30.280649926144758 204  Coherente /E>
Processing time was 6.83040976524353 seconds
30.428360413589363 205  Coherente /E>
Processing time was 6.699290990829468 seconds
30.576070901033972 206  incoherente /E>
Processing time was 8.068650960922241 seconds
30.72378138847858 207  incoherente /E>
Processing time was 8.049453020095825 seconds
30.87149187592319 208  Coherente /E>
Processing time was 7.7198145389556885 seconds
31.0192023633678 209  Coherente /E>

Processing time was 7.390418529510498 seconds
43.870014771048744 296  incoherente /E>
Processing time was 7.0056915283203125 seconds
44.01772525849335 297  Coherente /E>
Processing time was 8.180529594421387 seconds
44.16543574593796 298  Coherente /E>
Processing time was 7.219558000564575 seconds
44.31314623338257 299  Coherente /E>
Processing time was 8.449178457260132 seconds
44.46085672082718 300  Incoherente /E>
Processing time was 7.4786460399627686 seconds
44.608567208271786 301  incoherente /E>
Processing time was 8.198124170303345 seconds
44.7562776957164 302  Coherente /E>
Processing time was 7.497920513153076 seconds
44.903988183161005 303  Coherente /E>
Processing time was 7.616046667098999 seconds
45.05169867060561 304  Coherente /E>
Processing time was 8.787057876586914 seconds
45.19940915805022 305  Incoherente /E>
Processing time was 8.754936933517456 seconds
45.34711964549483 306  Coherente /E>
Processing time was 8.617862224578857 seconds
45.49483013293944 307  Incohe

Processing time was 8.690389633178711 seconds
58.34564254062038 394  Incoherente /E>
Processing time was 7.61903977394104 seconds
58.493353028064995 395  Coherente /E>
Processing time was 7.509116172790527 seconds
58.6410635155096 396  Coherente /E>
Processing time was 7.531101942062378 seconds
58.78877400295421 397  Coherente /E>
Processing time was 8.129430055618286 seconds
58.93648449039882 398  Coherente /E>
Processing time was 7.393126010894775 seconds
59.084194977843424 399  Coherente /E>
Processing time was 7.745485782623291 seconds
59.23190546528804 400  Coherente /E>
Processing time was 8.595093965530396 seconds
59.37961595273264 401  Incoherente /E>
Processing time was 7.535092115402222 seconds
59.527326440177255 402  Coherente /E>
Processing time was 7.657654762268066 seconds
59.67503692762186 403  Coherente /E>
Processing time was 8.368942022323608 seconds
59.822747415066466 404  Incoherente /E>
Processing time was 7.86207914352417 seconds
59.97045790251108 405  Coherente /

Processing time was 7.683257579803467 seconds
72.96898079763663 493  incoherente /E>
Processing time was 8.863075017929077 seconds
73.11669128508125 494  Coherente /E>
Processing time was 8.470650672912598 seconds
73.26440177252584 495  Coherente /E>
Processing time was 9.09343409538269 seconds
73.41211225997046 496  Coherente /E>
Processing time was 9.183857679367065 seconds
73.55982274741507 497  Coherente /E>
Processing time was 8.816735029220581 seconds
73.70753323485968 498  Coherente /E>
Processing time was 8.683859825134277 seconds
73.85524372230428 499  Coherente /E>
Processing time was 9.88270378112793 seconds
74.00295420974889 500  Coherente /E>
Processing time was 7.700008153915405 seconds
74.1506646971935 501  Coherente /E>
Processing time was 7.630218744277954 seconds
74.2983751846381 502  Coherente /E>
Processing time was 8.456376314163208 seconds
74.44608567208272 503  Coherente /E>
Processing time was 8.765891790390015 seconds
74.59379615952733 504  Incoherente /E>
Proc

Processing time was 7.880879878997803 seconds
87.59231905465288 592  Incoherente /E>
Processing time was 9.784574747085571 seconds
87.74002954209749 593  Incoherente /E>
Processing time was 7.654860258102417 seconds
87.8877400295421 594  Coherente /E>
Processing time was 7.822119235992432 seconds
88.0354505169867 595  Coherente /E>
Processing time was 7.311152219772339 seconds
88.18316100443131 596  Coherente /E>
Processing time was 7.573320388793945 seconds
88.33087149187593 597  Coherente /E>
Processing time was 9.704177618026733 seconds
88.47858197932054 598  incoherente /E>
Processing time was 7.775699615478516 seconds
88.62629246676514 599  Coherente /E>
Processing time was 8.840683460235596 seconds
88.77400295420975 600  Incoherente /E>
Processing time was 7.97983193397522 seconds
88.92171344165436 601  Coherente /E>
Processing time was 7.666171312332153 seconds
89.06942392909896 602  Coherente /E>
Processing time was 7.519260406494141 seconds
89.21713441654357 603  Coherente /E>

In [22]:
responses = [" ".join(x.split()[-2:]) for x in o.split("\n") if "Processing" not in x and x != ""]
responses = [[i, x] for i, x in enumerate(responses)]

In [23]:
test_v4 = test.copy()
test_v4["bloomz"] = [x[1] for x in responses]
test_v4["bloomz-label"] = [1 if "incoherente" in x[1].lower() else 0 for x in responses]
test_v4

,id,Q_id,Q,A,label,bloomz,bloomz-label
0,85815,4581,¿Qué aprendiste de la actividad del viernes?,ha ver las cajas porque las cajas chicas eran ...,0,Coherente /E>,0
1,64812,3535,¿Cómo creen ustedes que se reproducen las célu...,El proceso de división de una célula en dos cé...,0,Coherente /E>,0
2,84148,4491,¿ Qué frase para mejorar le darías tú a un am...,debes practicar los ejerccios,0,Coherente /E>,0
3,88565,4734,¿Qué aprendiste en la actividad del día viernes?,Nose porque no fui. A donde fueron ustedes😛😛😛😛😜,1,Coherente /E>,0
4,51688,2944,Diego compró 2 sopaipillas a $120 cada una. Pa...,las sopaipillas,1,Coherente /E>,0
...,...,...,...,...,...,...,...
672,54685,3109,Lorena dice que al multiplicar 3x0 le da como...,estamal poreque es 0,0,Incoherente /E>,1
673,87324,4665,Alicia comió 2/9 de un chocolate. Ella dice qu...,esta mal Alicia por que la cantidad es 7/8 de...,0,Coherente /E>,0
674,73798,3946,El cuarto básico visitará el zoológico. Si son...,nose\n,1,Incoherente /E>,1
675,95242,5093,Durante la sesión de conecta ideas María Jesús...,"es correcto, porque yo lo \nmultiplique por 4...",0,675 No,0


In [24]:
test_v4.to_excel("bloomz_label_v4.xlsx")

In [25]:
from sklearn.metrics import classification_report

In [26]:
print(classification_report(test_v4["label"], test_v4["bloomz-label"], digits=4))

              precision    recall  f1-score   support

           0     0.8480    0.7320    0.7857       541
           1     0.3095    0.4779    0.3757       136

    accuracy                         0.6809       677
   macro avg     0.5787    0.6050    0.5807       677
weighted avg     0.7398    0.6809    0.7034       677



In [27]:
q, a, label = train.sample(1).iloc[0]["Q A label".split()]
o = bloomz(few_shot_prompt_v4(str(q).replace("\n", ""), str(a).replace("\n", ""), shots_v4), max_length=8, temperature=0.4)[0]['generated_text']
bloomz_label = 1 if "incoherente" in o.lower() else 0
print(q, a, bloomz_label, label)

NameError: name 'train' is not defined